# Query AODN parquet dataset:

Using a python class ```GetAodn``` to:
- retrieve as a dictionnary all metadata from all parquet dataset available on a S3 bucket 
- search for dataset containing specific variable/attributes ....
- retrieve data from a dataset with time, bounding box and scalar filter as a panda dataframe

In [1]:
# only run once, then restart session if needed
!pip install uv

import os
import sys

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    os.system('uv pip install --system -r https://raw.githubusercontent.com/aodn/aodn_cloud_optimised/main/notebooks/requirements.txt')
else:
    os.system('uv venv')
    os.system('uv pip install -r https://raw.githubusercontent.com/aodn/aodn_cloud_optimised/main/notebooks/requirements.txt')

In [2]:
import requests
import os
if not os.path.exists('parquet_queries.py'):
  print('Downloading parquet_queries.py')
  url = 'https://raw.githubusercontent.com/aodn/aodn_cloud_optimised/main/aodn_cloud_optimised/lib/ParquetDataQuery.py'
  response = requests.get(url)
  with open('parquet_queries.py', 'w') as f:
      f.write(response.text)


In [3]:
from parquet_queries import GetAodn, create_time_filter, create_bbox_filter, query_unique_value, plot_spatial_extent, get_spatial_extent, get_temporal_extent, get_schema_metadata
import pyarrow.parquet as pq
import pyarrow.dataset as pds
import pyarrow as pa
import os
import pandas as pd
import pyarrow.compute as pc
import boto3
from fuzzywuzzy import fuzz
from botocore import UNSIGNED
from botocore.client import Config
from functools import lru_cache

# GetAodn Class

## Examples to use class

In [4]:
aodn_instance = GetAodn()

### Retrieving all parquet dataset information

In [5]:
# Creating a subinstance to use caching capability (maybe there's a cleaner way)
aodn_meta = aodn_instance.get_metadata()  
aodn_meta.metadata_catalog()

### Find all dataset containing specific information (such as temperature)

In [6]:
aodn_meta.find_datasets_with_attribute('temp', target_key='standard_name' )

### Get SOOP XBT data

In [7]:
aodn_instance.get_dataset('vessel_xbt_realtime_nonqc').get_temporal_extent()

In [8]:
aodn_instance.get_dataset('vessel_xbt_realtime_nonqc').plot_spatial_extent()

In [9]:
df = aodn_instance.get_dataset('vessel_xbt_realtime_nonqc').get_data(date_start='2023-01-31 10:14:00', 
                                                   date_end='2024-02-01 07:50:00',
                                                   lat_min=-34, lat_max=-32, lon_min=150, lon_max=155)

In [10]:
df[df['TEMP_quality_control'] == 1].sort_values('TIME').plot.scatter(x='TEMP', y='DEPTH', c='TIME',                                                                      
                                                                     cmap='RdYlBu_r', marker='.', linestyle="None").invert_yaxis()

In [11]:
df

### Get anmn hourly timeseries

In [12]:
aodn_instance = GetAodn()

In [13]:
aodn_instance.get_dataset('mooring_hourly_timeseries_delayed_qc').get_temporal_extent()

In [14]:
aodn_instance.get_dataset('mooring_hourly_timeseries_delayed_qc').plot_spatial_extent()

### Filter hourly timeseries for site BMP070

In [15]:
df = aodn_instance.get_dataset('mooring_hourly_timeseries_delayed_qc').get_data(date_start='2010-01-01 10:14:00', date_end='2024-02-01 07:50:00',
                                                                  scalar_filter={"site_code": "BMP070"})                                               

In [16]:
df

### Create a "climatology" for BPM070 at NOMINAL_DEPTH=52

Create a new variable **TEMP_CLIMATOLOGY** which is the daily mean over the available years.

In [17]:
# create a new variable DAY_MONTH in order to filter the data later on..
df['DAY_MONTH'] = df['TIME'].dt.strftime('%m-%d')

# Group by day and month, calculate daily mean temperature
daily_mean = df.groupby('DAY_MONTH')['TEMP'].mean().reset_index()
daily_mean.columns = ['DAY_MONTH', 'TEMP_CLIMATOLOGY']

# Merge back to original DataFrame on DAY_MONTH
df = pd.merge(df, daily_mean, on='DAY_MONTH', how='left')

# Drop the intermediate column 'DAY_MONTH'
df.drop('DAY_MONTH', axis=1, inplace=True)

#### Plot the TEMP variable and TEMP_CLIMATOLOGY together

#### Non-Interactive plot

In [18]:
import matplotlib.pyplot as plt

# Plot TEMP and TEMP_CLIMATOLOGY
plt.figure(figsize=(10, 6))
plt.plot(df['TIME'], df['TEMP'], label='TEMP', color='blue', alpha=0.7)
plt.plot(df['TIME'], df['TEMP_CLIMATOLOGY'], label='TEMP_CLIMATOLOGY', color='lightcoral', alpha=0.5)

# Fill between TEMP and TEMP_CLIMATOLOGY
plt.fill_between(df['TIME'], df['TEMP'], df['TEMP_CLIMATOLOGY'], color='gray', alpha=0.3)

plt.xlabel('Time')
plt.ylabel('Temperature')
plt.legend()

# Show the plot
plt.show()

#### Interactive plot with hvplot

In [19]:
import hvplot.pandas  # Import hvplot extension for pandas DataFrame
import holoviews as hv

In [ ]:
df = df[df["NOMINAL_DEPTH"] == 52].sort_values('TIME')

In [ ]:
# Create hvplot line plots for TEMP and TEMP_CLIMATOLOGY
temp_plot = df.hvplot.line(x='TIME', y='TEMP', label='TEMP', color='blue', ylabel='Temperature', xlabel='Time')
climatology_plot = df.hvplot.line(x='TIME', y='TEMP_CLIMATOLOGY', label='TEMP_CLIMATOLOGY', color='lightcoral')

overlay = (temp_plot * climatology_plot)

# Create a shaded area between the two line plots
filled_area = hv.Area((df['TIME'], df['TEMP'], df['TEMP_CLIMATOLOGY']), vdims=['TEMP', 'TEMP_CLIMATOLOGY']).opts(alpha=0.3, color='gray')

# Combine the line plot overlay with the filled area
combined_plot = (overlay * filled_area).opts(width=800, height=400, responsive=True)

# Show the combined plot
combined_plot